###Chromeインストール

In [2]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

In [3]:
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

!apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
!apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
!apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

Executing: /tmp/apt-key-gpghome.Ja7DliXyl5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.NNorlY0asK/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.20lvaFXCu0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

###selenium インストール

In [ ]:
!pip install selenium

###edinetから過去１年分の四半期決算データをダウンロード

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
import os
import shutil

files = os.listdir('.')
for file in files:
  if file.find('Xbrl') == -1:
    continue
  if os.path.isfile(file):
    os.remove(file)
  if os.path.isdir(file):
    shutil.rmtree(file)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)

target_url = "https://disclosure2.edinet-fsa.go.jp/"
driver.get(target_url)
time.sleep(3)

element = driver.find_element(By.ID, "W0018vD_KEYWORD")
#element.send_keys("エディア")
#element.send_keys("ロコンド")
#element.send_keys("セプテーニ")
word = "日機装"
word = "6619"
element.send_keys(word)
time.sleep(3)

select_elem = driver.find_element(By.ID, "W0018vD_KIKAN")
Select(select_elem).select_by_visible_text("過去１年")
time.sleep(3)

button = driver.find_element(By.ID, "W0018BTNBTN_SEARCH")
button.click()
time.sleep(5)

table = driver.find_element(By.ID, "GridContainerTbl")
trs = table.find_elements(By.TAG_NAME, 'tr')
for tr in trs[1:]:
    try:
      tds = tr.find_elements(By.TAG_NAME, 'td')
      print(tds[0].text)
      print(tds[1].text)
      tds[6].find_element(By.TAG_NAME, 'a').click()
    except:
      continue
    time.sleep(3)

2023/03/31 11:37
有価証券報告書－第18期(2022/01/01－2022/12/31)
2022/11/14 10:29
四半期報告書－第18期第3四半期(2022/07/01－2022/09/30)
2022/10/19 16:05
訂正四半期報告書－第18期第2四半期(2022/04/01－2022/06/30)
2022/10/19 16:03
訂正四半期報告書－第18期第1四半期(2022/01/01－2022/03/31)
2022/10/19 16:01
訂正有価証券報告書－第17期(2021/01/01－2021/12/31)
2022/10/19 16:00
訂正四半期報告書－第17期第3四半期(2021/07/01－2021/09/30)
2022/08/15 13:33
四半期報告書－第18期第2四半期(2022/04/01－2022/06/30)
2022/10/19 16:05
└　訂正四半期報告書－第18期第2四半期(2022/04/01－2022/06/30)
2022/05/13 10:40
四半期報告書－第18期第1四半期(2022/01/01－2022/03/31)
2022/10/19 16:03
└　訂正四半期報告書－第18期第1四半期(2022/01/01－2022/03/31)


###zipファイル解凍

In [8]:
import os
import zipfile

path = "."

# ファイル一覧を取得
files = os.listdir(path)
for file in files:
  if file.find('.zip') == -1:
    continue
  
  try:
      # zipファイル解凍
      extract_dir_name = file.replace('.zip', '')
      with zipfile.ZipFile(file) as existing_zip:
          existing_zip.extractall(extract_dir_name)
  except:
    continue

###４半期短期決算の営業利益データを取得

####・zip解凍フォルダからhtmファイルを取得 → 本年度と前年度の営業利益を取得

In [9]:
import glob

def get_values(xbrl_dir_name):
  # zip解凍フォルダからhtmファイルを取得
  files = os.listdir(xbrl_dir_name)
  dir_name = [file for file in files if os.path.isdir(xbrl_dir_name + '/' + file)][0]

  dir_path = f'{xbrl_dir_name}/{dir_name}'
  file_dir = f'{dir_path}/XBRL/PublicDoc/*'
  files = glob.glob(file_dir)
  html_files = [file for file in files if file.find('.htm') > 0]
  return get_html_values(html_files)

In [10]:
from bs4 import BeautifulSoup

def get_html_values(html_files):
  term = ""
  value = ""
  pre_term = ""
  pre_value = ""
  for html_file in html_files:
    # 本年度と前年度の営業利益を取得
    try:
      bs = BeautifulSoup(open(html_file, 'r').read(), 'lxml')
      if bs.find('p').text.find('四半期連結損益計算書') != -1:
        trs = bs.find_all('table')[0].find_all('tr')
        for tr in trs:
          if tr.text.find("四半期連結累計期間") > -1:
            terms = tr.text.strip().split("\n")
            terms.remove('')
            term = terms[-1]
            pre_term = terms[0]
          if tr.text.find("営業利益") > -1:
            values = tr.text.strip().split("\n")
            values.remove('')
            value = values[-1]
            pre_value = values[1]
    except:
      continue
    
    if term != '' and value != '' and pre_term != '' and pre_value != '':
      continue
    
    try:  # セプテーニなど
      h4s = [p for p in bs.find_all('h4') if p.text.find('四半期連結財政状態計算書') != -1]
      if len(h4s) > 0:
          for i in h4s[0].next_elements:
              if str(i)[0:3].find('tr') != -1:
                  tr = i
                  if value == '' and pre_value == '':
                    if tr.text.find("営業利益") > -1:
                        values = tr.text.strip().replace('\xa0', '').split("\n")
                        values = [v for v in values if len(v) > 0]
                        value = values[-1]
                        pre_value = values[1]

                  if term == '' and pre_term == '':
                    if tr.text.find("四半期連結累計期間") > -1:
                        values = [td.text for td in tr.find_all('td')]
                        values = [v for v in values if len(v) > 0 and v.find('\n\n') == -1]      
                        term = values[-1]
                        pre_term = values[1]
    except:
      continue

    if term != '' and value != '' and pre_term != '' and pre_value != '':
      continue
    
    try:  # ロコンドなど
      ps = [p for p in bs.find_all('h5') if p.text.find('損益計算書') != -1] + [p for p in bs.find_all('h4') if p.text.find('損益計算書') != -1]
      if len(ps) > 0:
        for p in ps:
          for i in p.next_elements:
              if str(i)[0:3].find('tr') != -1:
                  tr = i
                  if value == '' and pre_value == '':
                    if tr.text.find("営業利益") > -1:
                        values = tr.text.strip().split("\n")
                        values = [v for v in values if len(v) > 0]
                        value = values[-1]
                        pre_value = values[1]
                  if value == '' and pre_value == '':
                    if tr.text.find("営業損失") > -1:
                        values = tr.text.strip().split("\n")
                        values = [v for v in values if len(v) > 0]
                        value = values[-1]
                        pre_value = values[1]
                  if term == '' and pre_term == '':
                    if tr.text.find("自") > -1 and tr.text.find("至") > -1:
                        values = [td.text for td in tr.find_all('td')]
                        values = [v.replace('\n','') for v in values if len(v) > 0 and v.find('\n\n') == -1]
                        term = values[-1]
                        pre_term = values[0]
                      
    except:
      continue

  return (term, value, pre_term, pre_value)

In [11]:
import pandas as pd

path = "."
files = os.listdir(path)
extract_dir_names = [file_name for file_name in files if file_name.find('.zip') == -1 and file_name.find('Xbrl') != -1]

dic ={}
for extract_dir_name in extract_dir_names:
  # zip解凍フォルダからhtmファイルを取得 → 本年度と前年度の営業利益を取得
  term, value, pre_term, pre_value = get_values(extract_dir_name)

  if term == "":
    continue
 
  dic[term] = {'value': value, 'pre_value': pre_value}

dic 

{'当第３四半期連結累計期間(自\xa02021年１月１日\u3000至\xa02021年９月30日)': {'value': '422',
  'pre_value': '△3,354'},
 '当第２四半期連結累計期間(自\xa02022年１月１日\u3000至\xa02022年６月30日)': {'value': '2,755',
  'pre_value': '△107'},
 '当第１四半期連結累計期間(自\xa02022年１月１日\u3000至\xa02022年３月31日)': {'value': '439',
  'pre_value': '162'},
 '当事業年度(自\xa02022年１月１日\u3000至\xa02022年12月31日)': {'value': '△175',
  'pre_value': '△459'},
 '当第３四半期連結累計期間(自\xa02022年１月１日\u3000至\xa02022年９月30日)': {'value': '5,227',
  'pre_value': '422'},
 '当連結会計年度(自\xa02021年１月１日\u3000至\xa02021年12月31日)': {'value': '1,898',
  'pre_value': '△2,837'}}

###取得した４半期決算の営業利益をグラフ化

In [12]:
import re

# 取得したデータをデータフレーム化
df = pd.DataFrame(dic.values(), index=dic.keys())
df.columns = ['value', 'pre_value']

df = df.query("pre_value != '\xa0'")

# 営業利益のカラムを数値化
df = df.replace(',', '', regex=True)
df = df.replace('', '0', regex=True)
df = df.replace('△', '-', regex=True)
df["value"] = df['value'].astype('float')
df["pre_value"] = df['pre_value'].astype('float')

# 四半期の名前を正規化
try:
  df['title'] = [re.search('第(\d*)四半期', x).group(0) for x in df.index]
  df.index = df['title']
except:
  df['title'] = df.index #[re.search('第(\d*)四半期', x).group(0) for x in df.index]
  df.index = df['title']

del df['title'] #df = df.drop('title', axis=1)

# ソートして確認
df = df.sort_index()

df_total = pd.DataFrame({'value': [df['value'].sum()], 'pre_value': [df['pre_value'].sum()]}, index=["通期"])
df = pd.concat([df, df_total])

df

,value,pre_value
当事業年度(自 2022年１月１日 至 2022年12月31日),-175.0,-459.0
当第１四半期連結累計期間(自 2022年１月１日 至 2022年３月31日),439.0,162.0
当第２四半期連結累計期間(自 2022年１月１日 至 2022年６月30日),2755.0,-107.0
当第３四半期連結累計期間(自 2021年１月１日 至 2021年９月30日),422.0,-3354.0
当第３四半期連結累計期間(自 2022年１月１日 至 2022年９月30日),5227.0,422.0
当連結会計年度(自 2021年１月１日 至 2021年12月31日),1898.0,-2837.0
通期,10566.0,-6173.0


In [13]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='前年度', x=df.index, y=df['pre_value']),
    go.Bar(name='本年度', x=df.index, y=df['value'])
])

fig.update_layout(barmode='group')
fig.show()